In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [13]:
df = pd.read_csv("washington_rentals.csv")

features = ["Max Beds", "Max Baths", "Max Square Feet"]

target = "Max Rent Price"

non_seattle_df = df[df["City"] != "Seattle"]
non_seattle_correlation = non_seattle_df[features].corrwith(non_seattle_df[target])

seattle_df = df[df["City"] == "Seattle"]
seattle_correlation = seattle_df[features].corrwith(seattle_df[target])


combined_correlation = pd.DataFrame({
    "Non-Seattle Correlation": non_seattle_correlation,
    "Seattle Correlation": seattle_correlation
})

display(combined_correlation)

,Non-Seattle Correlation,Seattle Correlation
Max Beds,0.540207,0.205031
Max Baths,0.655208,0.180650
Max Square Feet,0.495112,0.232551


In [26]:

# Features and target
features = ["Max Beds", "Max Baths", "Max Square Feet"]
target = "Max Rent Price"

# Add a column to indicate whether each row has missing values
df['Used In Model'] = ~df[features + [target]].isnull().any(axis=1)

# Filter out rows with missing values in features or target
filtered_df = df.dropna(subset=features + [target])

# Prepare the feature matrix (X) and target vector (y)
X = filtered_df[features]
y = filtered_df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the coefficients, mean squared error, and R^2 score
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)
print("Mean squared error:", mse)
print("R^2 score:", r2)

# Merge predictions back with the initial DataFrame
test_indices = X_test.index
df.loc[test_indices, 'Predicted Max Rent Price'] = y_pred

# Display the DataFrame with the predictions
display(df)

Coefficients: [1.36027265e+02 8.34585918e+02 2.47928149e-01]
Intercept: 483.83361510175337
Mean squared error: 1426250.3914730167
R^2 score: 0.3774761953993452


,Unnamed: 0,Property ID,URL,Property Type,Address,City,State,ZIP Code,Country Code,Latitude,Longitude,Rental ID,Max Beds,Max Baths,Max Square Feet,Max Rent Price,Description,Used In Model,Predicted Max Rent Price
0,0,22700195,/WA/Auburn/Watermark/apartment/22700195,5,31655 Military Rd S,Auburn,WA,98001.0,1,47.318273,-122.285212,880f05a9-2c72-4c14-bff5-f9352e5a81f3,4,2.0,1342.0,2467.0,Seize your chance to live in a brand new commu...,True,NaN
1,1,209269,/WA/Auburn/4903-S-283rd-Pl-98001/home/209269,6,4903 S 283rd Pl,Auburn,WA,98001.0,1,47.347206,-122.271890,bee76375-7820-4480-b0d2-f9c8206ec450,4,3.0,2650.0,4295.0,Welcome to this beautiful home in the highly d...,True,NaN
2,2,40265097,/WA/Auburn/27810-47th-Pl-S-98001/home/40265097,6,27810 47th Pl S,Auburn,WA,98001.0,1,47.352111,-122.274841,9100b73e-2cf1-4ffe-af01-3af2fa5a1126,3,2.5,1930.0,3147.0,Offered By: Spinnaker Property Management Desc...,True,NaN
3,3,113179480,/WA/Auburn/3071-S-276th-Ct-98001/home/113179480,6,3071 S 276th Ct,Auburn,WA,98001.0,1,47.353470,-122.293626,efca4e59-e3f8-4636-a0ef-f6de4cf2d2a0,4,2.5,2623.0,4200.0,Avail Now. And just in time to enjoy summer wi...,True,NaN
4,4,173657113,/WA/Auburn/5638-S-302nd-St-98001/home/173657113,6,5638 S 302nd St,Auburn,WA,98001.0,1,47.331213,-122.264241,19fd27a2-39d9-4481-8d12-47d7dd218599,4,2.5,2472.0,3750.0,Welcome Home! This Space is sprawling with lig...,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4859,7606,126590848,/WA/Walla-Walla/601-E-Alder-St-99362/home/1265...,4,601 E Alder St,Walla Walla,WA,99362.0,1,46.067891,-118.326461,428a8602-7435-4426-869d-bef141eb79bd,1,1.0,458.0,895.0,NaN,True,NaN
4861,7608,126759534,/WA/Walla-Walla/1987-Alco-Ave-99362/home/12675...,4,1987 Alco Ave,Walla Walla,WA,99362.0,1,46.044910,-118.339810,741ce24d-14d7-4686-9d01-f34d7855ba10,3,2.0,1080.0,1795.0,NaN,True,NaN
4862,7609,126555038,/WA/Walla-Walla/1800-Evergreen-St-99362/apartm...,5,1800 Evergreen St,Walla Walla,WA,99362.0,1,46.073318,-118.308702,11bf147e-22ef-4a7e-8e62-4ee945687ded,2,1.0,712.0,1150.0,NaN,True,1766.998904
4863,7610,126381245,/WA/Walla-Walla/536-Pleasant-St-99362/home/126...,6,536 Pleasant St,Walla Walla,WA,99362.0,1,46.061948,-118.323622,0677417e-fe29-4265-bf5c-9f4e0fb87e01,4,2.0,1725.0,2495.0,Charming 4 bedroom Home<BR><BR>Welcome to this...,True,NaN
